In [ ]:
import pandas as pd
import json
import re
import shutil
import os
import unidecode

In [ ]:
def evolVal(valI, valE):
    if (valE != 0 and valI != valE):
        return valI - valE
    return 0

def evolPercent(ev, val):
    if val != 0: 
        return round((ev / val)*100, 2) 

def get_last_data(dff):
    most_recent_date = dff['period_date'].max()
    dfDate = dff.loc[dff.period_date == most_recent_date]
    return [most_recent_date, dfDate["valeur"].sum()]
    
def get_evolution(dff, last_date, last_value):
    dfevol = pd.DataFrame()
    dfevol = dff.copy() 
    dfevol.drop(dfevol.loc[dfevol['period_date'] == last_date].index, inplace=True)
    previous_last_data = get_last_data(dfevol)
    evol = evolVal(last_value, previous_last_data[1])
    evol_percent = evolPercent(evol, previous_last_data[1])
    return [evol, evol_percent]

def get_data_history(dff):
    dates = dff.sort_values(by="period_date").period_date.unique()
    values = []
    for date in dates:
        value = dff.loc[dff.period_date == date]
        values.append({"date": date.astype(str), "value": value["valeur"].sum()})
        
    return values 

def get_level(df, level, code_level):
    last_data = get_last_data(df)
    evolution = get_evolution(df, last_data[0], last_data[1])
    data_history = get_data_history(df)
    data_level = {}
    data_level["level"] = level
    data_level["code_level"] = code_level
    data_level["last_value"] = last_data[1]
    data_level["last_date"] = str(last_data[0]) #.to_pydatetime()
    data_level["evol"] = evolution[0]
    data_level["evol_percentage"] = evolution[1]
    data_level["evol_color"] = "red"
    data_level["values"] = data_history
    return data_level

def clean_folder(folder):
    # Suppression des fichiers générés précédemment
    try:
        shutil.rmtree(folder)
    except:
        pass
    os.makedirs(folder)

def clean_file(file_name):
    # Suppression du fichier généré précédemment
    try:
        os.remove(file_name)
    except:
        pass
    
def append_to_file(data, file_name):
    with open(file_name, "a", encoding="utf8") as output_file:
        json.dump(data, output_file, ensure_ascii=False)
        output_file.write('\n')

In [ ]:
col = ["indicateur", "period_date", "valeur", "mesure", "short_indic", "indic_id", "dep", "reg"]
df = pd.read_csv("pp_dep.csv", usecols=col, sep=";")
df['period_date'] = pd.to_datetime(df['period_date'])

file_name = 'france-relance-data-tableau-de-bord.txt'
clean_file(file_name)

indicateurs = df.indicateur.unique()
for indicateur in indicateurs:
    df_indicateur = df.loc[df.indicateur == indicateur]
        
    data = {}
    
    data["code"] = df_indicateur["indic_id"].iloc[0]
    data["nom"] = df_indicateur["short_indic"].iloc[0]
    data["unite"] = df_indicateur["short_indic"].iloc[0]
    
    # France
    france = get_level(df_indicateur, "nat", "fra") 
    
    data["france"] = [france]
    
    # Régions
    regions_data = []
    regions = df_indicateur.reg.unique()
    for region in regions:
        df_indicateur_region = df_indicateur.loc[df.reg == region]
        region_data = get_level(df_indicateur_region, "reg", region)
        regions_data.append(region_data)
    
    data["regions"] = regions_data
    
    # Départements
    departements_data = []
    departements = df_indicateur.dep.unique()
    for departement in departements:
        df_indicateur_departement = df_indicateur.loc[df.dep == departement]
        departement_data = get_level(df_indicateur_departement, "dep", departement)    
        departements_data.append(departement_data)
    
    data["departements"] = departements_data

    append_to_file(data, file_name)
    

In [ ]:
# Gestion des données nationales (données non présente au niveau départementale)
col = ["indicateur", "period_date", "valeur", "mesure", "short_indic", "indic_id", "dep", "reg"]
df = pd.read_csv("pp_nat.csv", usecols=col, sep=";")
df['period_date'] = pd.to_datetime(df['period_date'])

file_name = 'france-relance-data-tableau-de-bord.txt'


indicateurs = df.indicateur.unique()
for indicateur in indicateurs:
    df_indicateur = df.loc[df.indicateur == indicateur]
        
    data = {}

    data["code"] = df_indicateur["indic_id"].iloc[0]
    data["nom"] = df_indicateur["short_indic"].iloc[0]
    data["unite"] = df_indicateur["short_indic"].iloc[0]
    
    # France
    france = get_level(df_indicateur, "nat", "fra") 
    
    data["france"] = [france]

    append_to_file(data, file_name)
    